## Create _P.generosa_ primary gene annotations mapping file

See this [GitHub Issue(https://github.com/RobertsLab/resources/issues/1443).

This notebook will utilize NCBI BLAST and DIAMOND BLAST annotations generated by our [GenSas _P.generosa_ genome annotation](https://robertslab.github.io/sams-notebook/2019/09/28/Genome-Annotation-Pgenerosa_v074-a4-Using-GenSAS.html).

It will compare the two sets of SwissProt ID annotations (SPIDs) to determine lowest E-value and use that entry as the representative entry for a gene. It will then use that canonical list of SPIDs to pull gene names and gene ontology (GO) IDs from UniProt, and create a tab-deltimited annotation mapping file.

### List computer specs

In [ ]:
%%bash
echo "TODAY'S DATE"
date
echo "------------"
echo ""
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "
hostname
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

### Set variables
- `%env` indicates a bash variable
- without `%env` is Python variablec

In [ ]:
######################################################################
### Set directories
%env data_dir=/home/sam/data/P_generosa/genomes
%env analysis_dir=/home/sam/analyses/20220406-pgen-gene_annotation_mapping
analysis_dir="/home/sam/analyses/20220406-pgen-gene_annotation_mapping"

#####################################################################
### Input files
%env base_url=https://gannet.fish.washington.edu/Atumefaciens/20190928_Pgenerosa_v074.a4_gensas_annotation
%env blast_annotations=Panopea-generosa-vv0.74.a4.5d951a9b74287-blast_functional.tab
%env diamond_annotations=Panopea-generosa-vv0.74.a4.5d951bcf45b4b-diamond_functional.tab

######################################################################
### Output files
# UniProt batch output
%env uniprot_output=20220406-pgen-uniprot_batch-results.txt

# Gene name list for UniProt batch submission
%env spid_list=Panopea-generosa-v1.0.a4-blast-diamond-functional-SPIDs.txt

%env blast_diamond_cat=Panopea-generosa-v1.0.a4-blast-diamond-functional.tab

# Parsed UniProt
%env parsed_uniprot=20220406-pgen-accession-gene_name-gene_description-go_ids.csv

# Final output
%env joined_output=20220406-pgen-gene-gene_name-gene_description-go_ids.csv

######################################################################

### Programs

# UniProt batch submission/retrieval script
%env uniprot_mapping_script=/home/sam/programs/uniprot_mapping.pl

### Make input/output directories

In [ ]:
%%bash
# If directories don't exist, make them
mkdir --parents "${data_dir}" "${analysis_dir}"

### Download and inspect annotation files

`--quiet`: Prevents `wget` output from overwhelming Jupyter Notebook

`--continue`: If download was previously initiated, will continue where leftoff and will not create a second file if one already exists.

In [ ]:
%%bash
cd "${data_dir}"

wget --quiet --continue ${base_url}/${blast_annotations}
wget --quiet --continue ${base_url}/${diamond_annotations}

ls -ltrh

head -n 25 *.tab

### Count number of header lines (i.e. beginning with a `#`

In [ ]:
%%bash
cd "${data_dir}"

grep -c "^#" "${blast_annotations}" "${diamond_annotations}"

### Concatenate annotation files

Also modifies mRNA names to generate gene names instead.

- `awk 'NR > 13'`: Skips first 13 header lines
- `sort -k1,1 -k9,9`: Sorts on first field (mRNA name), then on 9th field (e-value)
- `sed 's/^21910-//'`: Removes leading info from each mRNA name, at the beginning of each line (`^`)
- `sed 's/.m0[0-9]//'`: Removes `.m0N` from each mRNA name.
- `awk '!array[$1]++'`: `awk` array that only prints line if it's the first occurrence of gene name (first field; `$1` (i.e. no duplicates)

In [ ]:
%%bash
cd "${data_dir}"

for file in ${blast_annotations} ${diamond_annotations}
do
    awk 'NR > 13' ${file}
done \
| sort -k1,1 -k9,9 \
| sed 's/^21910-//' \
| sed 's/.m0[0-9]//' \
| awk '!array[$1]++'\
>> "${blast_diamond_cat}"

wc -l "${blast_diamond_cat}"

head -n 25 "${blast_diamond_cat}"

### Create list os SwissProt IDs

In [ ]:
%%bash
cd "${analysis_dir}"

awk '{print $5}' "${data_dir}"/"${blast_diamond_cat}" > "${spid_list}"

wc -l "${spid_list}"

head "${spid_list}"

### Batch submission/retrieval to/from UniProt

Perl script obtained from UniProt: https://www.uniprot.org/help/api_batch_retrieval

Modified to accept file with list of IDs and to map SPID to UniProt Accession

In [ ]:
%%bash
# Print script for viewing
cat "${uniprot_mapping_script}"

%%bash
cd "${analysis_dir}"

# Run UniProt Prel mapping script and time how long it takes
time \
perl "${uniprot_mapping_script}" "${spid_list}" > "${uniprot_output}"

ls -ltrh

echo ""
echo ""
echo "--------------------------------------------------"
echo ""
echo "Line count:"

wc -l "${uniprot_output}"

echo "--------------------------------------------------"

### Check mapping output

In [ ]:
%%bash
cd "${analysis_dir}"

head -n 30 "${uniprot_output}"

### Parse the stuff we want

- UniProt accession

- Gene name/abbreviation

- Gene description

- GO IDs

#### Check DE descriptor lines to decide pattern matching

Checks lines beginning with `DE` to identify values in the 2nd field with `Name` in them.

Identifies unique values. This will determine how to parse properly after this.

In [ ]:
%%bash
cd "${analysis_dir}"

grep "^DE" "${uniprot_output}" | awk '$2 ~ /Name/ { print $2 }' | sort -u

In [ ]:
%%bash
cd "${analysis_dir}"
time \
while read -r line
do
  # Get record line descriptor
  descriptor=$(echo "${line}" | awk '{print $1}' )

  # Capture second field for evaluation
  go_line=$(echo "${line}" | awk '{print $2}')

  # Append GO IDs to array
  if [[ "${go_line}" == "GO;" ]]; then
    go_id=$(echo "${line}" | awk '{print $3}')
    go_ids_array+=("${go_id}")
  elif [[ "${go_line}" == "GeneID;" ]]; then
    # Uses sed to strip trailing semi-colon
    gene_id=$(echo "${line}" | awk '{print $3}' | sed 's/;$//')
  fi

  # Get gene description
  if [[ "${descriptor}"  == "DE" ]] && [[ "${go_line}"  == "RecName:" ]]; then
    # Uses sed to strip trailing spaces at end of line and remove commas
    gene_description=$(echo "${line}" | awk -F"[={]" '{print $2}' | sed 's/[[:blank:]]*$//' | sed 's/,//g')

  # Get alternate name
  elif [[ "${descriptor}"  == "DE" ]] && [[ "${go_line}"  == "AltName:" ]]; then
    # Uses sed to strip trailing spaces at end of line and remove commas
    alt_gene_description=$(echo "${line}" | awk -F"[={]" '{print $2}' | sed 's/[[:blank:]]*$//' | sed 's/,//g')

  # Get gene name
  elif [[ "${descriptor}"  == "GN"  ]] && [[ $(echo "${line}" | awk -F "=" '{print $1}') == "GN   Name" ]]; then
    # Uses sed to strip trailing spaces at end of line
    gene=$(echo "${line}" | awk -F'Name=|{' '{print $2}' | sed 's/[[:blank:]]*$//')

  # Get UniProt accession
  elif [[ "${descriptor}"  == "AC" ]]; then
    # Uses sed to strip trailing semi-colon
    accession=$(echo "${line}" | awk '{print $2}' | sed 's/;$//')

  # Identify beginning on new record
  elif [[ "${descriptor}"  == "//" ]]; then

    # Prints other comma-separated variables, then GOID1;GOID2;GOIDn
    # IFS prevents spaces from being added between GO IDs
    # sed removes ";" after final GO ID
    
    # If alt_gene_description is empty, print without that, otherwise print with that content.
    if [ -z "${alt_gene_description}" ]; then
      (IFS=; printf "%s,%s,%s,%s,%s\n" "${accession}" "${gene_id}" "${gene}" "${gene_description}" "${go_ids_array[*]}" | sed 's/;$//')
    else
      (IFS=; printf "%s,%s,%s,%s,%s,%s\n" "${accession}" "${gene_id}" "${gene}" "${gene_description}" "${alt_gene_description}" "${go_ids_array[*]}" | sed 's/;$//')
    fi

    # Re-initialize variables
    accession=""  
    descriptor=""
    gene=""
    gene_description
    gene_id=""
    go_id=""
    go_ids_array=()
  fi


done < "${uniprot_output}" >> "${parsed_uniprot}"